In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import h5py
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN","NAME"])


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T9","T13","T12","T2","T25","T14","T29","T15","T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")
    
# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()    


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
vc = application_df["CLASSIFICATION"].value_counts()
vc[vc>1]



C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = vc.index[vc < 1000].values

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()  

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=["APPLICATION_TYPE","CLASSIFICATION","AFFILIATION","USE_CASE","ORGANIZATION",
"INCOME_AMT","SPECIAL_CONSIDERATIONS"])
application_df


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values


C:\Users\Rohan\AppData\Local\Temp\ipykernel_1968\2152546080.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df.drop(["IS_SUCCESSFUL"],1).values


In [11]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
#ipython-input-10-f199198450cb>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
X = application_df.drop(["IS_SUCCESSFUL"],1).values


C:\Users\Rohan\AppData\Local\Temp\ipykernel_1968\1319857815.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df.drop(["IS_SUCCESSFUL"],1).values


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_of_input_features = len(X_train[0])
hidn_nodes_layer1 = 130
hidn_nodes_layer2 = 100
hidn_nodes_layer3 = 70
hidn_nodes_layer4 = 40

nn = tf.keras.models.Sequential()



In [14]:
# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidn_nodes_layer1, input_dim=number_of_input_features, activation="tanh")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidn_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidn_nodes_layer3, activation="LeakyReLU"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidn_nodes_layer4, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 130)               5720      
                                                                 
 dense_1 (Dense)             (None, 100)               13100     
                                                                 
 dense_2 (Dense)             (None, 70)                7070      
                                                                 
 dense_3 (Dense)             (None, 40)                2840      
                                                                 
 dense_4 (Dense)             (None, 1)                 41        
                                                                 
Total params: 28,771
Trainable params: 28,771
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=120)


Epoch 1/120
804/804 [==============================] - 9s 5ms/step - loss: 0.5667 - accuracy: 0.7247
Epoch 2/120
804/804 [==============================] - 4s 5ms/step - loss: 0.5563 - accuracy: 0.7308
Epoch 3/120
804/804 [==============================] - 4s 5ms/step - loss: 0.5530 - accuracy: 0.7315
Epoch 4/120
804/804 [==============================] - 4s 5ms/step - loss: 0.5519 - accuracy: 0.7307
Epoch 5/120
804/804 [==============================] - 4s 5ms/step - loss: 0.5500 - accuracy: 0.7334
Epoch 6/120
804/804 [==============================] - 4s 5ms/step - loss: 0.5490 - accuracy: 0.7329
Epoch 7/120
804/804 [==============================] - 4s 5ms/step - loss: 0.5475 - accuracy: 0.7338
Epoch 8/120
804/804 [==============================] - 4s 5ms/step - loss: 0.5472 - accuracy: 0.7328
Epoch 9/120
804/804 [==============================] - 4s 5ms/step - loss: 0.5470 - accuracy: 0.7350
Epoch 10/120
804/804 [==============================] - 4s 5ms/step - loss: 0.5460 - accura

804/804 [==============================] - 4s 5ms/step - loss: 0.5388 - accuracy: 0.7383
Epoch 82/120
804/804 [==============================] - 4s 5ms/step - loss: 0.5366 - accuracy: 0.7385
Epoch 83/120
804/804 [==============================] - 4s 5ms/step - loss: 0.5365 - accuracy: 0.7375
Epoch 84/120
804/804 [==============================] - 4s 5ms/step - loss: 0.5359 - accuracy: 0.7369
Epoch 85/120
804/804 [==============================] - 4s 5ms/step - loss: 0.5381 - accuracy: 0.7372
Epoch 86/120
804/804 [==============================] - 4s 4ms/step - loss: 0.5377 - accuracy: 0.7352
Epoch 87/120
804/804 [==============================] - 4s 4ms/step - loss: 0.5364 - accuracy: 0.7391
Epoch 88/120
804/804 [==============================] - 4s 4ms/step - loss: 0.5355 - accuracy: 0.7392
Epoch 89/120
804/804 [==============================] - 4s 5ms/step - loss: 0.5368 - accuracy: 0.7387
Epoch 90/120
804/804 [==============================] - 4s 4ms/step - loss: 0.5360 - accuracy: 

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 1s - loss: 0.5630 - accuracy: 0.7255 - 864ms/epoch - 3ms/step
Loss: 0.5630198121070862, Accuracy: 0.7254810333251953


In [18]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimisation2.h5")
